In [ ]:
from TTS.api import TTS
tts = TTS('tts_models/multilingual/multi-dataset/xtts_v2').to('cuda')

In [ ]:
from typing import List
import numpy as np

def convert_speech_to_audio_buffer(phrase:str, target_array:List[bytes]):
    wf = tts.tts(
        text = phrase,
        language = 'en',
        speaker_wav = 'hagrid_speech.wav',
        split_sentences = False,
        speed = 2.0,
    )
    buf = np.asarray(wf, dtype=np.float32) * 0.1
    target_array.append(buf)

In [ ]:
import ollama

model_id = 'hagrid'
try:
    ollama.delete(model_id)
except Exception:
    pass
ollama.create(model_id, path=f"{model_id}.modelfile")

In [ ]:
import re
import threading
import time
import ollama
import pyaudio

p = pyaudio.PyAudio()
OUT_STREAM = p.open(rate=23500, channels=1, format=pyaudio.paFloat32, output=True)

sentence_end_pattern = re.compile(r'[!\?\.]')
action_pattern = re.compile(r'\*[\s\w\d\-]+\*')

context = []
while True:
    prompt = input('>> ')
    if len(prompt) == 0:
        break
    context.extend([{'role':'user', 'content':prompt}])
    while len(context) > 8:
        del context[0]

    OUT_STREAM.stop_stream()
    phrase = []
    full_resp = []
    threads = []
    target_array = []
    for resp in ollama.chat(model=model_id, messages=context, stream=True):
        token = resp['message']['content']
        phrase.append(token)
        if sentence_end_pattern.search(token):
            speech = ''.join(phrase)
            speech_cleaned = action_pattern.sub('', speech)
            t = threading.Thread(target=convert_speech_to_audio_buffer, name='background_tts', kwargs={'phrase':speech, 'target_array':target_array})
            t.start()
            threads.append(t)
            full_resp.extend(phrase)
            phrase = []

    OUT_STREAM.start_stream()
    for bytes_val in target_array:
        OUT_STREAM.write(bytes_val)
    
    response = ''.join(full_resp)
    context.extend([{'role':'assistant', 'content':response}])
    
    print('---')
    print(prompt)
    print(response)